In [ ]:
from dotenv import load_dotenv
load_dotenv()
from typing import TypedDict, Annotated
from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages


In [ ]:
llm = init_chat_model("google_genai:gemini-2.5-flash")

In [ ]:
class State(TypedDict):
    message: Annotated[list, add_messages]

def chatbot(state: State) -> State:
    return {"message":llm.invoke(state["message"])}


builder = StateGraph(State)

builder.add_node("chatbot_node", chatbot)

builder.add_edge(START, "chatbot_node")
builder.add_edge("chatbot_node", END)

graph = builder.compile()

In [ ]:
from IPython.display import display, Image

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
message = {"role": "user", "content": "Who was the first president of India, name only?"}

response = graph.invoke({"message":[message]})
print("AI:", response['message'][-1].content)

In [ ]:
state = None
while True:
    in_message = input("your message: ")
    if in_message.lower() in  ["exit", "quit"]:
        break
    if state is None:
        state: State = {
            "message": [{"role": "user", "content": in_message}]
        }
    else:
        state["message"].append({"role": "user", "content": in_message})

    state = graph.invoke(state)
    print("Bot:", state["message"][-1].content)